# Проект 5. Компьютер говорит «Нет»
## Цель проекта: Разаработать скоринговую модель предсказания возможности дефолта клиетнов банка

## Задачи:
* Подготовить датасет к анализу EDA
* Выявить зависимости между отдельными признаками датасета
* Выявить вес наиболее значимых признаков
* создать модель и обучить ее на обучающем датасете
* По метрикам качества оценить качество модели


## import

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import auc, roc_auc_score, roc_curve, multilabel_confusion_matrix
import pandas_profiling
from sklearn.preprocessing import Normalizer, normalize
from collections import Counter
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

import warnings

In [4]:
RANDOM_SEED = 42

In [5]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

## DATA

In [6]:
DATASETS_LOC = ('../input/sf-dst-scoring')


In [7]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:

df_train = pd.read_csv(f'{DATASETS_LOC}/train.csv')
df_test = pd.read_csv(f'{DATASETS_LOC}/test.csv')
sample_submission = pd.read_csv(f'{DATASETS_LOC}/sample_submission.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../input/sf-dst-scoring/train.csv'

## Functions

In [ ]:
def subset_by_iqr(df, column, whisker_width=1.5):
    """Remove outliers from a dataframe by column, including optional
       whiskers, removing rows for which the column value are
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    Args:
        df (`:obj:pd.DataFrame`): A pandas dataframe to subset
        column (str): Name of the column to calculate the subset from.
        whisker_width (float): Optional, loosen the IQR filter by a
                               factor of `whisker_width` * IQR.
    Returns:
        (`:obj:pd.DataFrame`): Filtered dataframe
    """
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    low_outlier = q1 - whisker_width*iqr
    high_outlier = q3 + whisker_width*iqr
    column_median = df[column].median()

    # Apply filter with respect to IQR, including optional whiskers
    # else mode()

    return df[column].apply(lambda x: x if (x > low_outlier and x < high_outlier) else column_median)

In [ ]:
def bin_plot(col):
    """Does plot the binary features visualisation"""
    labels = list(data.query('default == 1')[col].value_counts().index)

    total = data[col].value_counts()
    defaulters = data.query('default==1')[col].value_counts()

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, total, width,  label='Total')
    rects2 = ax.bar(x + width/2, defaulters, width, color = 'r', label='Defaulters')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Amount')
    ax.set_title('Defaulters 0/1 in feature {}'.format(col))
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    plt.show()
    print('Доля дефолта от общего количества','\n', defaulters/total)

In [ ]:
def hist_plot(col):
    plt.figure()
    sns.distplot(data[col][data['default'] == 1], kde = False, color= 'red', hist = True)
    sns.distplot(data[col][data['default'] == 0], kde = False, color= 'blue', hist = True)
    plt.title(col)
    plt.show()

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Default, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Значение признаков датасета

Feature | Value	| Type
--------|-------|-----
client_id|	id клиента|	Числовой
app_date|	дата подачи заявки|	Временной
education|	уровень образования	|Категориальный
sex|	пол заёмщика|	Бинарный
age	|возраст заёмщика|	Числовой
car	|флаг наличия автомобиля |	Бинарный
car_type |	флаг автомобиля-иномарки|	Бинарный
decline_app_cnt	|количество отказанных прошлых заявок|	Числовой
good_work|	флаг наличия «хорошей» работы|	Бинарный
score_bki|	скоринговый балл по данным из БКИ|	Числовой
bki_request_cnt|	количество запросов в БКИ|	Числовой
region_rating|	рейтинг региона|	Числовой
home_address|	категоризатор домашнего адреса |	Категориальный
work_address|	категоризатор рабочего адреса|	Категориальный
income|	доход заёмщика	|Числовой
sna	|связь заемщика с клиентами банка	|Категориальный
first_time	|давность наличия информации о заемщике	|Категориальный
foreign_passport|	наличие загранпаспорта|	Бинарный
sample    |  маркер выборки  |Бинарный
default	|наличие дефолта (ЦЕЛЕВОЙ ПРИЗНАК)	|Бинарный

In [ ]:
#pandas_profiling.ProfileReport(data)

Как видим, большинство признаков у нас требует очистки и предварительной обработки

## Предобработка и анализ данных EDA

### Временной признак 'app_date'
конвертируем данные признака 'app_date' в формат времени

In [ ]:
data.app_date = data.app_date.apply(lambda x: pd.to_datetime(x) if x!=None else x)

Сгенерируем несколько признаков для выявления зависимостей.

In [ ]:
data['month'] = data.app_date.dt.month
data['date'] = data.app_date.dt.day
data['dayofweek'] = data.app_date.dt.dayofweek

In [ ]:
data = data.drop(['app_date'], axis =1)  #удалим за ненадобностью

## Числовые признаки

In [ ]:
num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt',
           'income', 'score_bki', 'region_rating'] #Числовые данные */ без client_id

### Признак 'age'


Распределение возрастов в категориях плательщиков и неплательщиков

In [ ]:
hist_plot('age')

In [ ]:
sns.boxplot(x='default', y='age',  data=data, palette={0: "blue", 1: "red"})

In [ ]:
data.groupby(data['default'])['age'].describe()

### Признак 'decline_app_cnt'

In [ ]:
hist_plot('decline_app_cnt')

In [ ]:
sns.boxplot(x='default', y='decline_app_cnt',  data=data, palette={0: "blue", 1: "red"})


Распределение по количеству отказов в кредите в категориях плательщиков и неплательщиков

In [ ]:

data.groupby(data['default'])['decline_app_cnt'].describe()


In [ ]:
part = data.decline_app_cnt.value_counts()[0] #наибольшее количество строк с одним значением
gen = data.shape[0] #общее количество строк

print("часть строк с '0' значением  в признаке 'decline_app_cnt' составляет {:.2f}% от общего числа.".format(part/gen*100))


#### исходя из данных в признаке, где 83% тех, кто не имел отказа, а абсллютное большинство из остальных 17% имеют только 1 отказ в получении кредита, заменим признак на бинарный, где 0- отказов не было, а 1-отказы были. 

In [ ]:
data['decline_app_cnt'] = data['decline_app_cnt'].apply(lambda x: 1 if x >= 1 else 0)

In [ ]:
data.decline_app_cnt.value_counts() #Проверка

запишем этот признак в бинарный и уберем из числового

In [ ]:
num_cols.remove('decline_app_cnt')
num_cols #Проверка

### Признак 'bki_request_cnt'

In [ ]:
hist_plot('bki_request_cnt')

In [ ]:
sns.boxplot(x='default', y='bki_request_cnt',  data=data, palette={0: "blue", 1: "red"})

##### Распределение по количеству запросов в БКИ  в категориях плательщиков и неплательщиков

In [ ]:
data.groupby(data['default'])['bki_request_cnt'].describe()

### Признак 'income'

In [ ]:
hist_plot('income')

In [ ]:
sns.boxplot(x='default', y='income',  data=data, palette={0: "blue", 1: "red"})

##### Распределение по доходам в категориях плательщиков и неплательщиков

In [ ]:
data.groupby(data['default'])['income'].describe()

### Признак 'score_bki'

In [ ]:
hist_plot('score_bki')

In [ ]:
sns.boxplot(x='default', y='score_bki',  data=data, palette={0: "blue", 1: "red"})

##### Распределение по оценкам БКИ в категориях плательщиков и неплательщиков

In [ ]:
data.groupby(data['default'])['score_bki'].describe()

### Признак 'region_rating'

In [ ]:
hist_plot('region_rating')

In [ ]:
sns.boxplot(x='default', y='region_rating',  data=data, palette={0: "blue", 1: "red"})

##### Распределение по рейтингу регионов в категориях плательщиков и неплательщиков

In [ ]:
data.groupby(data['default'])['region_rating'].describe()

### Все числовые данные содержат выбросы и несбалансированы
### для корректой работы скоринговой модели заменим выбросы на медианные значения по каждому признаку

In [ ]:
for col in num_cols:
    data[col] = subset_by_iqr(data, col, whisker_width=1.5)

In [ ]:
data[num_cols].describe() # проверяем распределение числовых признаков

In [ ]:
data[num_cols].isna().sum() # NAN значения в числовых данных

### Значимость числовых переменных

In [ ]:
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

## Бинарные признаки

In [ ]:
bin_cols = ['decline_app_cnt','sex','car', 'car_type', 'good_work', 'foreign_passport']

In [ ]:
bin_plot('decline_app_cnt')

In [ ]:
bin_plot('sex')

In [ ]:
bin_plot('car')

In [ ]:
bin_plot('car_type')

In [ ]:
bin_plot('good_work')

In [ ]:
bin_plot('foreign_passport')

### Преобразуем бинарные признаки в числовые переменные

In [ ]:
for column in bin_cols:
    data[column] = LabelEncoder().fit_transform(data[column])

In [ ]:
 data[bin_cols].head() #убедимся в преобразовании

## Категориальные переменные


In [ ]:
cat_cols = ['education', 'home_address',
           'work_address', 'sna', 'first_time']

In [ ]:
print('Пропущеных значений {}, что составляет {:.2f}% от общего количества'.format(data.education.isna().sum(), data.education.isna().sum() /data.shape[0]*100))

что за данные в 'education'?

In [ ]:
data.education.value_counts()

Приведенная абривеатура может означать уровень образования, где

Graduation's code | Value
-----|-------------------------
SCH  |  school graduated
GRD  |  Graduate Research Degrees
UGR  |  Upper graduated researchers
PGR  |  Post graduated researchers
ACD  |  Upper post graduated degrees


##### Исходя из небольшого процента пропущенных данных, заменим пропущенные значения наиболее встречающимися в выборке

In [ ]:
data['education'] = data['education'].fillna(data['education'].mode()[0])

In [ ]:
data.isna().sum() # проверка

In [ ]:
bin_plot('education')

In [ ]:
#распределение данных в education
data['education'].value_counts()

#### преобразуем признак 'education' в численный формат

In [ ]:
data['education'] = LabelEncoder().fit_transform(data['education'])
data['education'].value_counts()

In [ ]:
bin_plot('home_address')

In [ ]:
bin_plot('work_address')

In [ ]:
bin_plot('sna')

In [ ]:
bin_plot('first_time')

### Целевой признак

In [ ]:
data.query('sample ==1').default.hist()
data.query('sample ==1').default.value_counts()


### Значимость категориальных и бинарных признаков

In [ ]:

imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
data.info()

### посмотрим вновь на таблицу корреляции 

In [ ]:
corr = data.corr()
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Heatmap")\
    .set_precision(2)\
    .set_table_styles(magnify())

#### выявлены сильные корреляции между:
    * car и car_type
    * home_address и work_address
    * client_id и month


In [ ]:
data.drop(['client_id','work_address','car_type'], axis = 1, inplace=True) # удаление сильно скоррелированных признаков

In [ ]:
cat_cols.remove('work_address')



In [ ]:
bin_cols.remove('car_type')

### смотрим, что получилось

In [ ]:
data.sample(10)

In [ ]:
data.info()

#### после проведения предобработки данных и анализа EDA разделим вновь датасет на тренировочную и тестовую выборки


In [ ]:
df_train =  data.query('sample == 1').drop(['sample'], axis=1)
df_test = data.query('sample == 0').drop(['sample'], axis=1)

y = df_train.default.values # наш таргет
df_train = df_train.drop(['default'], axis=1)

## Preprocessing

In [ ]:
X_cat =  OneHotEncoder(sparse = False).fit_transform(df_train[cat_cols].values)

#### Стандартизируем числовые переменные

In [ ]:
X_num = StandardScaler().fit_transform(df_train[num_cols].values)

In [ ]:
df_train.sample(3)

In [ ]:
df_test.sample(3)

#### И объединим стандартизованные числовые, бинарные и закодированные категориальные переменные в одно признаковое пространство, разделив при этом признаки и целевую переменную.


In [ ]:
X = np.hstack([X_num, df_train[bin_cols].values, X_cat])

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации.
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
df_test.shape, df_train.shape, X.shape, X_train.shape, X_test.shape

## Model

In [ ]:
lg = LogisticRegression(max_iter=1000)
lg.fit(X_train,y_train)
y_pred = lg.predict_proba(X_test)[:,1]

fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc =  roc_auc_score(y_test, y_pred)


## Оценка качества модели

In [ ]:
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

### на этом месте должны быть гипперпараметры :-)

In [ ]:
y_pred

#### Давайте найдем оптимальное значение параметра регуляризации для логистической регрессии.

In [ ]:
# Добавим типы регуляризации
penalty = ['l1', 'l2']

In [ ]:
# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)


In [ ]:

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(lg, hyperparameters, cv=5, verbose=0)

In [ ]:
best_model = clf.fit(X_train, y_train)

In [ ]:
# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])